In [2]:

"""" 
!pip install selenium
!pip install requests
!pip install beautifulsoup4
!
"""

'" \n!pip install selenium\n!pip install requests\n!pip install beautifulsoup4\n!\n'

In [3]:
import requests               
from bs4 import BeautifulSoup as bs
import json
import os
import re
import utils
import shutil

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

# For Chrome
#Selenium - Webdriver version 호환성
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait



In [4]:
"""
├── 110_공시서류확인및서명 
│   ├── 110_1_제도안내
│   ├── 110_1_업무절차
│   ├── 110_1_작성사례
├── 120_이사의경영진단및분석의견
├── 210 정기 보고서
├── 220 주요사항 보고서
│   ├── 220_1 제도안내
│   │   ├── 220_1-1_개요
│   │   └── 220_1-2_자기주식 취득, 처분 등
├── 230 발행공시
├── 240 합병
├── 310 대량 보유상황 보고
├── 320 임원 소유상황 보고
├── 330 단기매매차익 반환
├── 410 공시업무 스케줄
├── 500 공시의무 미 이행 안내 유의사항
├── 510 전자공시절차
├── 540 동영상 가이드
├── 550 공시관련부서안내
├── 기업공시제도 일반
├── 공시유의사항
├── FAQ
├── 보도자료
└── 단기매매차익 발생사실

class = page-tab 
class = nav nav-tabs role = 'tablist' 
href = #content-tab-1 (220_1 제도안내) 
href = #content-tab-2 (220_2 세부업무)
href = #content-tab-3 (220_3 Q&A)

class = tab-content
    |- id = content-tab-1-1 (220_1-1 개요)
    |- id = content-tab-1-2 (220_1-1 자기주식 취득 처분 등 )



"""

"\n├── 110_공시서류확인및서명 \n│   ├── 110_1_제도안내\n│   ├── 110_1_업무절차\n│   ├── 110_1_작성사례\n├── 120_이사의경영진단및분석의견\n├── 210 정기 보고서\n├── 220 주요사항 보고서\n│   ├── 220_1 제도안내\n│   │   ├── 220_1-1_개요\n│   │   └── 220_1-2_자기주식 취득, 처분 등\n├── 230 발행공시\n├── 240 합병\n├── 310 대량 보유상황 보고\n├── 320 임원 소유상황 보고\n├── 330 단기매매차익 반환\n├── 410 공시업무 스케줄\n├── 500 공시의무 미 이행 안내 유의사항\n├── 510 전자공시절차\n├── 540 동영상 가이드\n├── 550 공시관련부서안내\n├── 기업공시제도 일반\n├── 공시유의사항\n├── FAQ\n├── 보도자료\n└── 단기매매차익 발생사실\n\nclass = page-tab \nclass = nav nav-tabs role = 'tablist' \nhref = #content-tab-1 (220_1 제도안내) \nhref = #content-tab-2 (220_2 세부업무)\nhref = #content-tab-3 (220_3 Q&A)\n\nclass = tab-content\n    |- id = content-tab-1-1 (220_1-1 개요)\n    |- id = content-tab-1-2 (220_1-1 자기주식 취득 처분 등 )\n\n\n\n"

In [5]:
ROOT_PATH = '../DCM/dart/'
folder_names = [] 

def create_directory(save_path, link_text, link_href ):
    match = re.search(r'menu=(\d+)', link_href)
    if match:
        menu_number = match.group(1)
    else:
        # Use a default value or skip the entry if no menu number is found
        menu_number = '600'

    folder_name = f"{menu_number}_{link_text}"
    folder_names.append(folder_name)
    print(f"텍스트: {link_text}, 링크: {link_href}, 폴더명: {folder_name}")
    save_path =os.path.join(save_path, folder_name)
    os.makedirs(save_path, exist_ok=True)
    print(f"{save_path} 폴더 생성")
    return save_path


def create_directory(path):
    if not os.path.exists(path):
        print(f'## Make directory in {path}')
        os.makedirs(path)


# 'dart/' 폴더 내에서 '{url_num}_'으로 시작하는 디렉토리 확인
def find_directory(directory, url_num):
    if not os.path.exists(directory):
        return False  # 디렉토리가 없으면 False 반환
    
    for name in os.listdir(directory):
        full_path = os.path.join(directory, name)
        if os.path.isdir(full_path) and name.startswith(url_num):
            return full_path  # 조건에 맞는 디렉토리가 있으면 full_path 반환
        
    return None  # 찾지 못하면 None 반환


def make_dir_path(url_num:str, title:str, tab_num=None, chapter_num=None, subtitle=None):
    print(f"## url_num {url_num}, title {title}, tab_num {tab_num}, chapter_num {chapter_num}, subtitle {subtitle}")

    # 상위 디렉토리 (e.g. 110_공시서류확인및서명)
    if tab_num == None and chapter_num == None:
        path = os.path.join(ROOT_PATH, url_num + '_' + title)
        create_directory(path)
    
    # 하위 디렉토리 (e.g. 110_1_제도안내)
    elif tab_num != None and chapter_num == None:
        # url_num이 같은 상위 디렉토리 서칭하기
        parent_dir = find_directory(ROOT_PATH, url_num)
        if parent_dir != None:
            path = os.path.join(parent_dir, url_num + '_' + tab_num + '_' + title)
            create_directory(path)
        else:
            raise ValueError("No parent directory exists!")
    
    # 탭 안에 하위 탭 (이하 챕터) 있는 경우 (e.g. 220_1-1_제도안내_개요)
    # # ../DCM/dart/220_주요사항보고서/220_2-1_세부업무_Ⅰ 개요 
    else:
        # url_num이 같은 상위 디렉토리 서칭하기
        parent_dir = find_directory(ROOT_PATH, url_num)
        if parent_dir != None:
            path = os.path.join(parent_dir, url_num + '_' + tab_num + '-' + chapter_num  + '_' + title + '_' + subtitle)
            create_directory(path)
        else:
            raise ValueError("No parent directory exists!")

    return path


In [6]:


def html_parsing(html, url_num):
    soup = BeautifulSoup(html, 'html.parser')

    li_elements = soup.select("#contents > div.content > div.page-tab > ul > li")

    # 탭 단위
    for idx, li in enumerate(li_elements, start=1):
        tab_title = li.get_text(strip=True).strip().replace(' ', '')
        path = make_dir_path(url_num, tab_title, str(idx))
        content = soup.select_one(f"#content-tab-{idx}").prettify()
        
        sub_soup = BeautifulSoup(content, 'html.parser')

        sub_li_elements = sub_soup.select(f"#content-tab-{idx} > div.sub-top-tab > ul > li")

        if sub_li_elements:
            print("## 하위 탭 존재 ##")
            for sub_idx, sub_li in enumerate(sub_li_elements, start=1):
                subtab_title = sub_li.get_text(strip=True).strip().replace(' ', '')
                path = make_dir_path(url_num, tab_title, str(idx), str(sub_idx), subtab_title)
                print("## sub path loaded in")
                sub_content = soup.select_one(f"#content-tab-{idx}-{sub_idx}").prettify()

                with open(os.path.join(path, tab_title + "_" + subtab_title +".txt"), "w") as file:
                    print(f"## {path} 에 {os.path.join(path, tab_title + '_' + subtab_title +'.txt')} 저장 ##")
                    file.write(sub_content)
                
        else:
            with open(os.path.join(path, tab_title+".txt"), "w") as file:
                print(f"## {path} 에 {os.path.join(path, tab_title+'.txt')} 저장 ##")
                file.write(content)
    



In [14]:

def download_file(download_button, default_download_dir, path):
    try:
        
        # 다운로드 버튼 클릭
        download_button.click()
        time.sleep(2)  # 파일 다운로드를 위한 대기 시간

        # 다운로드된 파일 목록 출력 및 이동
        downloaded_file_name = os.listdir(default_download_dir)[0]
        print(f"다운로드된 파일: {downloaded_file_name}")
        downloaded_file_path  = os.path.join(default_download_dir, downloaded_file_name)  #eg., legal_graph/DCM/dart/220_주요사항보고서/220_1-1_개요/temp_downloads/공시진행.hwp
        move_file_path  = os.path.join(path,downloaded_file_name)
        #rename = os.path.join(default_download_dir, f"{downloaded_file.split('.')[0]}_{index}_{downloaded_file.split('.')[1]}"
        
        #같은 파일 있으면 다음 번호 붙여주기 
        index = 0 
        if os.path.exists(move_file_path):
            index += 1
            os.rename(downloaded_file_path, f"{downloaded_file_path.split('.')[0]}_{index}_{downloaded_file_path.split('.')[1]}")
        
        # 파일 이동
        shutil.move(downloaded_file_path, move_file_path)
        print(f"파일 {downloaded_file_path}을 {move_file_path}로 이동")

        # 다운로드 폴더 비우기 
        shutil.rmtree(default_download_dir)
    except Exception as e:
        print(f"파일 다운로드 실패: {e}")


In [24]:
# sub-top-tab 내부의 각 탭 순회
def process_sub_tab(driver, sub_tab, default_download_dir, path):
    print("sub tab 내부 ")
    #save_path = ../DCM/dart/220_주요사항보고서/220_1_제도안내

    print(" ## sub_tab ", sub_tab.text)
    print(" ## default_download_dir ", default_download_dir)
    print(" ## save_path ", path)
    
    driver.execute_script("arguments[0].click();", sub_tab)
    time.sleep(1)  # 페이지 로드 대기

    # SUB TAB 제목 추출 
    sub_tab_title = sub_tab.text
    # 다운로드 버튼 처리
    try:
        download_buttons = driver.find_elements(By.CSS_SELECTOR, ".tab-pane.active .btn-download")
        for i, download_button in enumerate(download_buttons):
            download_file(download_button, default_download_dir , path)
            
            
    except Exception as e:
        print(f"Sub-Tab '{sub_tab_title}' 다운로드 실패: {e}")
    
    


In [35]:

def menu_crawler(url, url_num, path):
    default_download_dir = os.path.abspath(os.path.join(path, "temp_downloads")) 
    #e.g.,  c:\Users\Shic\2_development\GIB\legal_graph\DCM\dart\110_공시서류확인및서명\temp_downloads 
    print(f"기본 다운로드 폴더: {default_download_dir}")  
    
    # 기본 다운로드 경로가 없으면 생성
    if os.path.exists(default_download_dir):
        shutil.rmtree(default_download_dir)
    os.makedirs(default_download_dir)

    # Chrome의 다운로드 경로 설정
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' : default_download_dir}
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": default_download_dir,  # 절대 경로 사용
        "download.prompt_for_download": False,  # 다운로드 확인 창 비활성화
        "download.directory_upgrade": True,  # 기존 디렉토리 사용 허용
        "safebrowsing.enabled": True , # 안전 브라우징 활성화
        'prefs': default_download_dir
    })

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options) 
    driver.get(url)
    time.sleep(2)  # 페이지 로드 대기


    tabs = driver.find_elements(By.CSS_SELECTOR, '.page-tab .nav-tabs li a') 
    tabs_len = len(tabs)
    print(f"{url_num}의 길이 : {tabs_len} ") # 220의 길이가 15? 

    for idx, tab in enumerate(tabs, start=1):
        driver.execute_script("arguments[0].click();", tab)
        tab_title = tab.text
        path = make_dir_path(url_num, tab_title, str(idx))

        sub_tabs = driver.find_elements(By.CSS_SELECTOR, ".tab-pane.active .sub-top-tab .nav-tabs li a")
        sub_tabs_len = len(sub_tabs)
        print(f"{url_num}의 하위 탭 길이 : {sub_tabs_len} ") # 220의 길이가 15? 

        if sub_tabs:
            print("## 하위 탭 존재 ##")
            for sub_idx, sub_tab in enumerate(sub_tabs, start=1):
                driver.execute_script("arguments[0].click();", sub_tab)
                subtab_title = sub_tab.text
                path = make_dir_path(url_num, tab_title, str(idx), str(sub_idx), subtab_title)
                print("## sub path loaded in")
                process_sub_tab(driver, sub_tab, default_download_dir, path)
            continue
                    
        else:
            print("## 하위 탭 없음 ##")    
            download_buttons = driver.find_elements(By.CSS_SELECTOR, ".tab-pane.active .btn-download")
            for download_button in download_buttons:
                download_file(download_button, default_download_dir, path)
            
        #다운로드 폴더 삭제
        #shutil.rmtree(default_download_dir)

In [28]:

options = Options()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL 설정
URL = "https://dart.fss.or.kr/info/selectGuide.do"


# 페이지 열기
driver.get(URL)
time.sleep(2)

# 메뉴 열기 버튼 클릭
menu_btn = driver.find_element(By.CLASS_NAME, 'btn-menu')
menu_btn.click()
time.sleep(1)


# 메뉴 별 폴더 생성 
#기존에 있는 경우 지우고 재생성 
if os.path.exists(ROOT_PATH):
    print(f"{ROOT_PATH} 폴더 삭제")
    shutil.rmtree(ROOT_PATH)
os.makedirs(ROOT_PATH, exist_ok=True)

folder_names = [] 

# ul li 안의 ul li 안의 a 태그를 가져오기
menu_list = driver.find_elements(By.CSS_SELECTOR, 'ul li ul li')

for idx, menu in enumerate(menu_list):
    a = menu.find_element(By.TAG_NAME, 'a')
    link_text = a.get_attribute('innerText').strip().replace(" ", "")
    link_href = a.get_attribute('href')

    if link_text:
        print(f"텍스트: {link_text}, 링크: {link_href}")
        try:
            url_num = re.findall(r'\d+', link_href)[0]
            path = make_dir_path(url_num, link_text) # 110_공시서류확인및서명     
        except:
            print(f"공시업무 게시판 제외")
            continue
        print("path", path)
        menu_crawler(link_href, url_num, path)

    else:
        print(f"빈 텍스트: 링크: {link_href}")

driver.quit()

../DCM/dart/ 폴더 삭제
텍스트: 공시서류확인및서명, 링크: https://dart.fss.or.kr/info/main.do?menu=110
## url_num 110, title 공시서류확인및서명, tab_num None, chapter_num None, subtitle None
## Make directory in ../DCM/dart/110_공시서류확인및서명
path ../DCM/dart/110_공시서류확인및서명
기본 다운로드 폴더: c:\Users\Shic\2_development\GIB\legal_graph\DCM\dart\110_공시서류확인및서명\temp_downloads


KeyboardInterrupt: 

In [18]:
from bs4 import BeautifulSoup
import requests
url = "https://dart.fss.or.kr/info/main.do?menu=220"
response = requests.get(url)

# 요청이 성공했는지 확인하기 위해 상태 코드를 출력할 수 있습니다
print(response.status_code)


soup = BeautifulSoup(response.content, 'html.parser')

print(soup.prettify())

200
<!DOCTYPE html>
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, user-scaleable=no" name="viewport"/>
  <meta content="ie=edge" http-equiv="X-UA-Compatible"/>
  <title>
   기업공시 길라잡이
  </title>
  <!-- Start // load library resource -->
  <link href="/info/resource/css/nanumgothic.css" rel="stylesheet"/>
  <!-- <link href="https://fonts.googleapis.com/css?family=Nanum+Gothic:regular,700,800" rel="preload"> -->
  <link href="/info/resource/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="/info/resource/css/slick.css" rel="stylesheet"/>
  <script src="/info/resource/js/jquery-3.3.1.min.js">
  </script>
  <script src="/info/resource/js/bootstrap.bundle.min.js">
  </script>
  <script src="/info/resource/js/slick.min.js">
  </script>
  <script src="/info/resource/js/jquery.scrollTo.min.js">
  </script>
  <script src="/info/resource/js/jquery.word-break-keep-all.min.js">
  </script>
  <!-- <scr

In [ ]:
#class="content-box tab-pane active show" 